General imports

In [1]:
import pandas as pd

Reading dataset

In [2]:
data = pd.read_csv('GlobalLandTemperaturesByCountry.csv')

Quick view to the data

In [3]:
data.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland
